In [1]:
from tensorflow.keras.models import load_model
from skimage import transform, io, exposure
from imutils import paths
import numpy as np
import imutils, random, cv2

In [10]:
model = load_model("output/trafficsignnet15.model")

In [11]:
# loading the label names
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

In [12]:
imagePaths = list(paths.list_images("gtsrb-german-traffic-sign/Test"))

In [13]:
random.shuffle(imagePaths)

In [14]:
imagePaths = imagePaths[:25]

In [15]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    
    # load the image, resize it to 32x32 pixels, and then apply
    # Contrast Limited Adaptive Histogram Equalization (CLAHE),
    # just like we did during training
    image = io.imread(imagePath)
    image = transform.resize(image, (32, 32))
    image = exposure.equalize_adapthist(image, clip_limit=0.1)
    
    # preprocess the image by scaling it to the range [0, 1]
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    
    # make predictions using the traffic sign recognizer CNN
    preds = model.predict(image)
    j = preds.argmax(axis=1)[0]
    label = labelNames[j]
    
    # load the image using OpenCV, resize it, and draw the label on it
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=128)
    cv2.putText(image, label, (5, 15), cv2.FONT_HERSHEY_SIMPLEX,
        0.45, (0, 0, 255), 2)
    
    # save the image to disk
    p = "example" + str(i) + ".png"
    cv2.imwrite(p, image)